In [38]:
import cv2
import numpy as np

p_objects = []
p_images = []

nx = 9
ny = 6
# x, y coordinates
p_object = np.zeros((ny*nx,3), np.float32)
p_object[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)



## calibration

In [415]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')
# Step through the list and search for chessboard corners
for fname in images:
    img = mpimg.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    if ret:
        p_images.append(corners)
        p_objects.append(p_object)
    
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(p_objects, p_images, gray.shape[::-1], None, None)


In [416]:
cal_images = glob.glob('./camera_cal/calibration*.jpg')
test_images = glob.glob('./test_images/*.jpg')
for fname in cal_images:
    img = mpimg.imread(fname)
    undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imwrite("output_images/calibration/"+os.path.basename(fname), cv2.cvtColor(undistorted_img, cv2.COLOR_RGB2BGR))
for fname in test_images:
    img = mpimg.imread(fname)
    undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imwrite("output_images/test_images_undistorted/"+os.path.basename(fname), cv2.cvtColor(undistorted_img, cv2.COLOR_RGB2BGR))

## Color and Gradient Transformation

In [418]:

def to_binary(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return color_binary, combined_binary
    

In [422]:
for fname in test_images:
    img = mpimg.imread(fname)
    undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
    color_binary, combined_binary = to_binary(undistorted_img)
    cv2.imwrite("output_images/test_images_binary/color/"+os.path.basename(fname), cv2.cvtColor(color_binary, cv2.COLOR_RGB2BGR))
    cv2.imwrite("output_images/test_images_binary/combined/"+os.path.basename(fname), cv2.normalize(combined_binary, None, 0, 255, cv2.NORM_MINMAX))

## Perspective Transform

In [426]:
import math
import numpy as np
from sklearn import linear_model
from statistics import median

def get_src(img):
    bottom_left_x = int(5.5/27.8*img.shape[1])
    bottom_left_y = int(15/15.7*img.shape[0])
    bottom_right_x = int(22.6/27.8*img.shape[1])
    bottom_right_y = int(15/15.7*img.shape[0])
    top_left_x = int(11.6/27.8*img.shape[1])
    top_left_y = int(10.7/15.7*img.shape[0])
    top_right_x = int(16.2/27.8*img.shape[1])
    top_right_y = int(10.7/15.7*img.shape[0])
    return np.float32([[top_left_x, top_left_y], [bottom_left_x, bottom_left_y], [bottom_right_x, bottom_right_y], [top_right_x, top_right_y]])

def get_dst(img):
    offset_x = 300
    offset_y = 50
    return np.float32([[offset_x, offset_y], [offset_x, img.shape[0]-offset_y],[img.shape[1]-offset_x, img.shape[0]-offset_y], [img.shape[1]-offset_x, offset_y]])

def to_bev(img):
    src = get_src(img)
    dst = get_dst(img)
    img_size = (img.shape[1], img.shape[0])
    M = cv2.getPerspectiveTransform(src, dst)
    M_reverse = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, img_size)
    return warped, M, M_reverse

In [427]:
for fname in test_images:
    img = mpimg.imread(fname)
    undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
    warped, M, M_reverse = to_bev(undistorted_img)
    cv2.imwrite("output_images/test_images_bev/"+os.path.basename(fname), cv2.cvtColor(warped, cv2.COLOR_RGB2BGR))


## Fit Polynominal

In [441]:

left_fit = np.empty([1,3])
right_fit = np.empty([1,3])
left_fit_real = np.empty([1,3])
right_fit_real = np.empty([1,3])
frame_count = 0
ym_per_pix = 30/720 
xm_per_pix = 3.7/700

def find_lane_pixels(binary_warped):
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    nwindows = 9
    margin = 100
    minpix = 50

    window_height = np.int(binary_warped.shape[0]//nwindows)
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    for window in range(nwindows):
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current-margin 
        win_xleft_high = leftx_current+margin 
        win_xright_low = rightx_current-margin 
        win_xright_high = rightx_current+margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        good_left_inds = ((nonzerox>=win_xleft_low)&(nonzerox<=win_xleft_high)&(nonzeroy>=win_y_low)&(nonzeroy<=win_y_high)).nonzero()[0]
        good_right_inds = ((nonzerox>=win_xright_low)&(nonzerox<=win_xright_high)&(nonzeroy>=win_y_low)&(nonzeroy<=win_y_high)).nonzero()[0]
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(img_shape, leftx, lefty, rightx, righty):
    global left_fit
    global right_fit
    global left_fit_real
    global right_fit_real

    left_fit_real = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_real = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    return left_fitx, right_fitx, ploty


def fit_polynomial_first_frame(binary_warped):
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    left_fitx, right_fitx, ploty = fit_polynomial(binary_warped.shape, leftx, lefty, rightx, righty)

    ## Visualization ##
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    fit_points_left = (np.asarray([left_fitx, ploty]).T).astype(np.int32)
    fit_points_right = (np.asarray([right_fitx, ploty]).T).astype(np.int32)
    cv2.polylines(out_img, [fit_points_left], False, (255, 255, 0), 3)
    cv2.polylines(out_img, [fit_points_right], False, (255, 255, 0), 3)

    return out_img, ploty, left_fitx, right_fitx

def search_around_poly(binary_warped):
    global frame_count

    margin = 100
    minpix = 450
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    prev_left_value = left_fit[0]*nonzeroy**2+left_fit[1]*nonzeroy+left_fit[2]
    prev_right_value = right_fit[0]*nonzeroy**2+right_fit[1]*nonzeroy+right_fit[2]
    left_lane_inds = ((nonzerox>=prev_left_value-margin)&(nonzerox<=prev_left_value+margin))
    right_lane_inds = ((nonzerox>=prev_right_value-margin)&(nonzerox<=prev_right_value+margin))
    
    #check points within margin
    if len(left_lane_inds) < minpix or len(right_lane_inds) < minpix:
        frame_count = 0

    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    leftx_current = np.int(np.mean(leftx))       
    rightx_current = np.int(np.mean(rightx))
    if abs(leftx_current-rightx_current)<500:
        return fit_polynomial_first_frame(binary_warped)
    left_fitx, right_fitx, ploty = fit_polynomial(binary_warped.shape, leftx, lefty, rightx, righty)
    
    ## Visualization ##
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window1 = np.array([np.stack((left_fitx-margin, ploty),axis=1)])
    # left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
    #                           ploty])))])
    left_line_window2 =np.array([np.flipud(np.stack((left_fitx+margin, ploty), axis=1))])

    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    fit_points_left = (np.asarray([left_fitx, ploty]).T).astype(np.int32)
    fit_points_right = (np.asarray([right_fitx, ploty]).T).astype(np.int32)
    cv2.polylines(out_img, [fit_points_left], False, (255, 255, 0), 3)
    cv2.polylines(out_img, [fit_points_right], False, (255, 255, 0), 3)
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    return result, ploty, left_fitx, right_fitx


In [437]:
for fname in test_images:
    img = mpimg.imread(fname)
    undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
    warped, M, M_reverse = to_bev(undistorted_img)
    color_binary, combined_binary = to_binary(warped)
    out_img, ploty, left_fitx, right_fitx = fit_polynomial_first_frame(combined_binary)
    cv2.imwrite("output_images/test_images_polyfit/"+os.path.basename(fname), cv2.cvtColor(out_img, cv2.COLOR_RGB2BGR))
    

## Calculate Curvature

In [442]:
def measure_curvature_pixels(ploty):
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return left_curverad, right_curverad

def measure_curvature_real_world(ploty):
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit_real[0]*y_eval*ym_per_pix + left_fit_real[1])**2)**1.5) / np.absolute(2*left_fit_real[0])
    right_curverad = ((1 + (2*right_fit_real[0]*y_eval*ym_per_pix + right_fit_real[1])**2)**1.5) / np.absolute(2*right_fit_real[0])
    
    return left_curverad, right_curverad

## Map lane area on image


In [443]:
def map_lane_area(undist):
    lane_img = np.zeros_like(undist)
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(lane_img, np.int_([pts]), (0, 255, 0))
    img_size = (undist.shape[1], undist.shape[0])
    lane_img_unwarped = cv2.warpPerspective(lane_img, M_reverse, img_size)
    result = cv2.addWeighted(undist, 1, lane_img_unwarped, 0.3, 0)

    line_img = np.zeros_like(undist)
    fit_points_left = (np.asarray([left_fitx, ploty]).T).astype(np.int32)
    fit_points_right = (np.asarray([right_fitx, ploty]).T).astype(np.int32)
    cv2.polylines(line_img, [fit_points_left], False, (255, 255, 0), 5)
    cv2.polylines(line_img, [fit_points_right], False, (255, 255, 0), 5)
    line_img_unwarped = cv2.warpPerspective(line_img, M_reverse, img_size)
    result = cv2.addWeighted(result, 0.8, line_img_unwarped, 0.5, 0)
    return result


In [440]:
for fname in test_images:
    img = mpimg.imread(fname)
    undistorted_img = cv2.undistort(img, mtx, dist, None, mtx)
    warped, M, M_reverse = to_bev(undistorted_img)
    color_binary, combined_binary = to_binary(warped)
    out_img, ploty, left_fitx, right_fitx = fit_polynomial_first_frame(combined_binary)
    result = map_lane_area(undistorted_img)
    cv2.imwrite("output_images/test_images_lane_area/"+os.path.basename(fname), cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
    

## run on video

In [444]:
   
def process_image(image):
    global frame_count
    undist = cv2.undistort(image, mtx, dist, None, mtx)
    b_color, b_gray = to_binary(undist)
    warped, M, M_reverse  = to_bev(b_gray)
    if frame_count>0:
        out_img, ploty, left_fitx, right_fitx = search_around_poly(warped)
    else:
        out_img, ploty, left_fitx, right_fitx = fit_polynomial_first_frame(warped)
    R_left, R_right = measure_curvature_pixels(ploty)
    frame_count += 1

    result = map_lane_area(undist)
    cv2.putText(result, f"left curvatur: {R_left:.2f}, right curvatur: {R_right:.2f}", (200, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, color = (255, 255, 255))
    return result

In [445]:
from moviepy.editor import VideoFileClip, vfx, clips_array
from IPython.display import HTML

video_output = 'project_video_output.mp4'
# clip1 = VideoFileClip("project_video.mp4").subclip(45,50)
clip1 = VideoFileClip("project_video.mp4")
frame_count = 0
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(video_output, audio=False)

Moviepy - Building video project_video_output.mp4.
Moviepy - Writing video project_video_output.mp4



Moviepy - Done !
Moviepy - video ready project_video_output.mp4
CPU times: user 19min 52s, sys: 28.6 s, total: 20min 20s
Wall time: 1min 29s


In [446]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))